# Leetcode
### Sharping coding skills
[Website](https://leetcode.com)


# Valid palindrome #125

[Exercise](https://leetcode.com/problems/valid-palindrome/)

Given a string, determine if it is a palindrome, considering only alphanumeric characters and ignoring cases.

**Note:** For the purpose of this problem, we define empty string as valid palindrome.

**Example 1:**

* **Input:** "A man, a plan, a canal: Panama"
* **Output:** true

**Example 2:**

* **Input:** "race a car"
* **Output:** false

 

**Constraints:**

`s` consists only of printable ASCII characters.



## Regex approach \w

It seems obvious that we should use a regex to exclude noisy chars from the string (everything else that is not a letter or number.
This first approach makes use of `\w` and convert the matched chars into a string.

It scored **60ms - 45%**

In [3]:
import re
# A string for testing
test_str = "a String with numb3rs, commas%, semicolons; and curly brackets{}"
s = "A man, a plan, a canal: Panama"

def isPalindrome(s):
    x = re.findall("\w", s)
    x = ''.join(x).lower()
    x = x.replace('_', '')  # \w allows the underscore

    half = len(x) // 2
    fwd, bwd = 0, len(x) - 1
    for _ in range(half):
        if x[fwd] != x[bwd]:
            return False
        fwd += 1
        bwd -= 1
    return True

In [4]:
%%timeit
isPalindrome(s)

4.76 µs ± 30.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Regex approach [0-9A-Za-z]
This approach refines above by:
* It's not necessary to convert back into a string 
* Replacig the regex form `\w` to `[0-9A-Za-z]` allows to get rid of replacement of the underscore

It scored **52ms - 67%**

In [5]:
import re
# A string for testing
test_str = "a String with numb3rs, commas%, semicolons; and curly brackets{}"
s = "A man, a plan, a canal: Panama"

def isPalindrome(s):
    x = re.findall("[0-9A-Za-z]", s)
    bwd = len(x) - 1
    for fwd in range(len(x) // 2):
        if x[fwd].lower() != x[bwd].lower():
            return False
        bwd -= 1
    return True
isPalindrome(s)

True

In [6]:
%%timeit
isPalindrome(s)

4.95 µs ± 31.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Regex without converting the entire string

This is more elaborated and almost 3 times less efficient than previous attempts. Thought that I could get rid of converting the entire string to a valid one at the expense of comparing each char and determining whether they are valid. This introduced extra complexity because the mid point had to be recalculated each time the loop hits a void char.

Submission returned **176ms & 5.27%**

In [7]:
import re
# A string for testing
test_str = "a String with numb3rs, commas%, semicolons; and curly brackets{}"
s = "A man, a plan, a canal: Panama"
def isPalindrome(s):
    # True positions in the original string, ie, including special chars
    fwd, bwd = 0, len(s) - 1
    
    # Relative dynamic values
    valid_len = len(s)  # this updates when excluding chars
    dynamic_middle = valid_len // 2  # starting point for middle
    dynamic_fwd = 0  # fwd position without the special chars
    valid_chars = "[0-9A-Za-z]"
    while dynamic_fwd != dynamic_middle:
        # first ensure the chars are valid
        test_fwd, test_bwd = [
            re.match(valid_chars, s[way]) for way in (fwd, bwd)]
        if test_fwd and test_bwd:
            # both chars are valid and we can compare them
            if s[fwd].lower() != s[bwd].lower():
                # they don't match, so break
                return False
            # otherwise advance one position in the chain
            fwd += 1
            bwd -= 1
            dynamic_fwd += 1
        if not test_fwd:
            # Foward char is forbidden
            valid_len -= 1
            dynamic_middle = valid_len // 2  # recalculate the middle point
            fwd += 1
        if not test_bwd:
            # Backward char is forbidden
            valid_len -= 1
            dynamic_middle = valid_len // 2  # recalculate the middle point
            bwd -= 1
    
    # If all the chars match the string is a palindrome
    return True
        
isPalindrome(s)

True

In [8]:
%%timeit
isPalindrome(s)

30.7 µs ± 232 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## `isalnum()` solution
It seems that one can use `isalnum()` string method instead of regex. Also, it's not necessary to calculate middle points, just simply keep the loop running while forward is below backward.

This scored slightly worse than Regex [0-9A-Za-z] approach, **56ms & 47%** but with better memory performance **14.3mb**

In [9]:
test_str = "a String with numb3rs, commas%, semicolons; and curly brackets{}"
s = "A man, a plan, a canal: Panama"
def isPalindrome(s):
    fwd, bwd = 0, len(s) - 1
    while fwd < bwd:
        while fwd < bwd and not s[fwd].isalnum():
            fwd += 1
        while fwd < bwd and not s[bwd].isalnum():
            bwd -= 1
        if s[fwd].lower() != s[bwd].lower():
            return False
        fwd += 1
        bwd -= 1
    return True
isPalindrome(s)

True

In [10]:
%%timeit
isPalindrome(s)

4.28 µs ± 36.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Regex substitution
We can make use of regex substitution directly and compare the string without even loop over by using `:` slice notation
This approach worked best to date: **36ms & 95%**

In [32]:
import re
test_str = "a String with_ numb3rs, commas%, semicolons; and curly brackets{}"
s = "A man, a plan,_ a canal: Panama"
def isPalindrome(s):
        if not s:
            return True
        s = re.sub('[^\w]', '', s)
        p = s.lower().replace('_', '')  # replace
        return p == p[::-1]
isPalindrome(s)

True

In [33]:
%%timeit
isPalindrome(s)

2.61 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


It seems that getting rid of the evaluation of `s` we can sqezee a bit the efficiency: **32ms & 98%.** It also turned out that `replace()` and its regex counterpart perform the same.

In [38]:
import re
test_str = "a String with_ numb3rs, commas%, semicolons; and curly brackets{}"
s = "A man, a plan,_ a canal: Panama"
def isPalindrome(s):
        s = re.sub('[_]|[^\w]', '', s)  # replace everything but alphanumeric
        p = s.lower()
        return p == p[::-1]
isPalindrome(s)

True

In [39]:
%%timeit
isPalindrome(s)

2.98 µs ± 20.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Palindrome linked list #234
**[exercise](https://leetcode.com/problems/palindrome-linked-list/)**  

Given a singly linked list, determine if it is a palindrome.

Example 1:

Input: 1->2
Output: false

Example 2:

Input: 1->2->2->1
Output: true

Follow up:
Could you do it in O(n) time and O(1) space?

---
Basically you have an input like this:
`{'val': 3, 'next': {'val': 5, 'next': None}}`

In [1]:
tests = [
    [0, 2, 3, 3, 2, 0],
    [0, 3, 3, 3, 2, 0],
    [0, 2, 3, 2, 0],
    [1, 2, 3, 4, 5],
    [0, 0],
    [-190,-190], 
]

class listNode:
    def __init__(self, val, next=None):
        self.val = val
        self.next = next

def create_ln(array):
    ln = listNode(array[0])

    for n in array[1: ]:
        ln = listNode(n, ln)
    return ln

ln = create_ln(tests[0])

## List approach
The following three approaches make use of a list the first is filled upwards and the last backwards. Then both are compared. When the first submission was made I thought that maybe comparing half of the arrays could improve the speed, however it was clearly worse especially on the last submission to leetcode (not noticeable in the notebook, both perform in 1.35µs & 1.34µs) maybe because of the tests applied there.

In [7]:
def isPalindrome(ln):
    """
    Build an array forward and another one backward and then compare both.
    
    Too computational expensive.    
    """
    fwd, bwd = list(), list()
    while ln:
        fwd.append(ln.val)
        bwd.insert(0, ln.val)
        ln = ln.next if ln.next else None
    return fwd == bwd

for array in tests:
    ln = create_ln(array)
    print('Case: {} => {}'.format(array, isPalindrome(ln)))

Case: [0, 2, 3, 3, 2, 0] => True
Case: [0, 3, 3, 3, 2, 0] => False
Case: [0, 2, 3, 2, 0] => True
Case: [1, 2, 3, 4, 5] => False
Case: [0, 0] => True
Case: [-190, -190] => True


In [6]:
%%timeit
isPalindrome(create_ln(array))

1.1 µs ± 15.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
def isPalindrome(ln):
    """
    Build an array forward and another one backward and then compare both.
    
    Let's compare just the half of the array. Performed sightly worse than
    previous one.   
    """
    fwd, bwd = list(), list()
    while ln:
        fwd.append(ln.val)
        bwd.insert(0, ln.val)
        ln = ln.next if ln.next else None
    split = int(1 + (len(fwd) // 2) if len(fwd) % 2 else len(fwd) / 2)
    return fwd[:split] == bwd[:split]

for array in tests:
    ln = create_ln(array)
    print('Case: {} => {}'.format(array, isPalindrome(ln)))

Case: [0, 2, 3, 3, 2, 0] => True
Case: [0, 3, 3, 3, 2, 0] => False
Case: [0, 2, 3, 2, 0] => True
Case: [1, 2, 3, 4, 5] => False
Case: [0, 0] => True
Case: [-190, -190] => True


In [9]:
%%timeit
isPalindrome(create_ln(array))

1.35 µs ± 3.37 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
def isPalindrome(ln):
    """
    Create an array where forward part will be stored from the half on and 
    backward part will be stored in the beginning.
    
    This is even worse than the previous: 1544ms!!
    """
    fwd = list()
    while ln:
        fwd.append(ln.val)
        fwd.insert(0, ln.val)
        ln = ln.next if ln.next else None
    half = len(fwd) // 2
    quarter = half // 2
    return fwd[:half][:quarter] == fwd[half:][:quarter]

for array in tests:
    ln = create_ln(array)
    print('Case: {} => {}'.format(array, isPalindrome(ln)))

Case: [0, 2, 3, 3, 2, 0] => True
Case: [0, 3, 3, 3, 2, 0] => False
Case: [0, 2, 3, 2, 0] => True
Case: [1, 2, 3, 4, 5] => False
Case: [0, 0] => True
Case: [-190, -190] => True


In [11]:
%%timeit
isPalindrome(create_ln(array))

1.34 µs ± 2.81 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Fast & slow approach
This solution proposed by [Stefan Pochmann](https://leetcode.com/problems/palindrome-linked-list/discuss/64500/11-lines-12-with-restore-O(n)-time-O(1)-space) is quite smart. It makes a reversed copy of the first half and compares one by one against the second half. How do we find the half? just double jump in linked list to reach the end in the half moves. 

It has a tricky part that has to do with python assignments: `rev, rev.next, slow = slow, rev, slow.next`. In this expression the right hand side is evaluated before making any assignment to the left. Then, one by one are assigned so, in the first loop, `rev` takes the value of `slow`, the original ln; `rev.next`, whose value has been updated to be equal to `slow.next`, changes its value to `None` the previous value stored for `rev`; finally `slow` is shifted to its next value.

The takeaway here is that in every loop, `rev` takes all the diminishing slow linked list but afterwards it drops all the values but the current one and replaces them by the already reversed array.

In [12]:
def isPalindrome(head):
    """
    Make a reversed version of the first half and then match against the second
    half.
    """
    rev = None
    slow = fast = head
    while fast and fast.next:
        # Jump two positions forward so, we'll run out of numbers to loop with
        # in the middle
        fast = fast.next.next
        rev, rev.next, slow = slow, rev, slow.next  # The tricky part
    if fast:
        # if the list is odd, there's still one more element in the middle
        slow = slow.next
    while rev and rev.val == slow.val:
        # Finally compare rev linked list and the remaining slow, the last half
        slow = slow.next
        rev = rev.next
    # if the loop reaches the end rev stops at None so flip the bool of rev 
    return not rev
for array in tests:
    ln = create_ln(array)
    print('Case: {} => {}'.format(array, isPalindrome(ln)))

Case: [0, 2, 3, 3, 2, 0] => True
Case: [0, 3, 3, 3, 2, 0] => False
Case: [0, 2, 3, 2, 0] => True
Case: [1, 2, 3, 4, 5] => False
Case: [0, 0] => True
Case: [-190, -190] => True


In [13]:
%%timeit
isPalindrome(create_ln(array))

895 ns ± 1.65 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


# Palindrome #9

**[Exercise](https://leetcode.com/problems/palindrome-number/)**

Determine whether an integer is a palindrome. An integer is a palindrome when it reads the same backward as forward.

Example 1:

Input: 121  
Output: true

Example 2:

Input: -121  
Output: false  
Explanation: From left to right, it reads -121. From right to left, it becomes 121-. Therefore it is not a palindrome.

Example 3:

Input: 10  
Output: false  
Explanation: Reads 01 from right to left. Therefore it is not a palindrome.

Follow up:

Coud you solve it without converting the integer to a string?


In [44]:
import numpy as np
def assertions():
    # define checks: input integer and expected output
    checks = (
        (121, True),
        (-121, False),
        (10, False),
        (1001, True),
        (0, True),
        (1234567890987654321, True)
    )
    
    failed = False
    for c in checks:
        E = isPalindrome(c[0])
        if E != c[1]:
            print('Expected outcome for {}: {}; returned: {}'.format(
                   c[0], c[1], E,))
            failed = True
            break
    if failed:
        return

## Length, list & loop
There are three steeps in this approach:
* Get the length of the integer
* Build a list out of that length by int division of powers of ten
* Test the positions in the list

In [51]:
def isPalindrome(i):
    # First, dismiss ints under 0
    if i < 0:
        return False
    
    # First thing to do is calculate the length of the int
    length = 0
    number = i
    while number != 0:
        length += 1
        number //= 10
        
    # With that number we can create a list of the digits by decimal division
    digits = list()
    exp = length - 1  # Since we want the zero exponent
    for _ in range(length):
        digit = i // 10**exp
        digits.append(digit)
        i -= digit*10**exp
        exp -= 1
        
    # Finally test the positions in the list
    for idx in range(len(digits) // 2):
        f, s = digits[idx], digits[-idx - 1]
        if f != s:
            return 0
    return 1

In [52]:
%%timeit
assertions()

19.6 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## LeetCode solution
Start out by excluding border cases:
* integers under 0
* integers ending in 0 (but not 0 itself)

Then reverse the first half of the integer, to do so:
* Notice that the reverse of $12=1\times 10^1 + 2\times 10^0$ is $21=2\times 10^1 + 1\times 10^0$ 
* Grow by ten fold the already reversed number and add current last digit
* Discard added digit from original number
* Repeat until reversed number is greater than the remaining part of original number

Also notice that when the original number's digits are odd, the central number can (and must) be excluded so the final comparison is either `remaining == reversed` or `remaining == reversed // 10` 


In [50]:
def isPalindrome(n):
    # First, dismiss ints under 0
    if n < 0 or (n % 10 == 0 and n != 0):
        return False
    rev = 0
    while n > rev:
        rev = rev * 10 + n % 10  # grow by ten fold and add current last digit 
        n //= 10  # discard added digit
    
    # Exclude central digit for odd number of digits
    return rev == n or n == rev // 10

In [49]:
%%timeit
assertions()

3.13 µs ± 38.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Reverse integer #7

**[Exercise](https://leetcode.com/problems/reverse-integer/)**

Given a 32-bit signed integer, reverse digits of an integer.

Example 1:

Input: 123  
Output: 321

Example 2:

Input: -123  
Output: -321

Example 3:

Input: 120  
Output: 21

Note:  
Assume we are dealing with an environment which could only store integers within the 32-bit signed integer range: $[−2^{31},  2^{31} − 1]$. For the purpose of this problem, assume that your function returns 0 when the reversed integer overflows.


In [19]:
import numpy as np
def assertions():
    # define checks: input integer and expected output
    checks = (
        (123, 321),
        (-123, -321),
        (120, 21),
        (214748364, 463847412),
        (2147483648, 0)  # Overflow
    )
    
    failed = False
    for c in checks:
        E = reverse(c[0])
        if E != c[1]:
            print('Expected integer for {}: {}; returned: {}'.format(
                   c[0], c[1], E,))
            failed = True
            break
    if failed:
        return

## Exponentiation approach
Multiply each digit in the int by growing powers of ten  

$123 = 1\times 10^2+2\times 10^1+3\times 10^0$  
$321 = 1\times 10^0+2\times 10^1+3\times 10^2$

It ranked quite nice, over 73% 

In [20]:
def reverse(i):
    # get the digits excluding the sign
    negative = i < 0
    digits = str(i)[1:] if negative else str(i)
    outcome = 0
    for e, d in enumerate(digits):
        outcome += int(d)*10**e
    
    # rescue the sign (if any)
    if negative:
        outcome *= -1
    
    # Overflows int32
    if outcome < -2**31 or outcome > 2**31-1:
        outcome = 0
        
    return outcome


In [21]:
%%timeit
assertions()

9.88 µs ± 33.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Running Sum of 1d Array (#1480)

**[Exercise](https://leetcode.com/problems/running-sum-of-1d-array/)**

Given an array `nums`. We define a running sum of an array as `runningSum[i] = sum(nums[0]…nums[i])`.

Return the running sum of `nums`.

 

Example 1:

Input: nums = [1,2,3,4]  
Output: [1,3,6,10]  
Explanation: Running sum is obtained as follows: [1, 1+2, 1+2+3, 1+2+3+4].  

Example 2:

Input: nums = [1,1,1,1,1]  
Output: [1,2,3,4,5]  
Explanation: Running sum is obtained as follows: [1, 1+1, 1+1+1, 1+1+1+1, 1+1+1+1+1].  

Example 3:

Input: nums = [3,1,2,10,1]  
Output: [3,4,6,16,17]  

 

Constraints:

* 1 <= nums.length <= 1000
* -10^6 <= nums[i] <= 10^6

In [10]:
import numpy as np
def assertions():
    # define checks: list, target, expected output
    checks = (
        ([1, 1, 1, 1], [1, 2, 3, 4,]),
        ([1, 10, 20, 7], [1, 11, 31, 38]),
        ([1, 2, 3, 4], [1, 3, 6, 10])
    )
    
    failed = False
    for c in checks:
        E = runningSum(c[0])
        if E != c[1]:
            print('Expected array for {}: {}; returned: {}'.format(
                   c[0], c[1], E,))
            failed = True
            break
    if failed:
        return

## Numpy.cumsum() approach
Maybe the shortest and easiest (but not the fastest) way to achieve it is the numpy's `cumsum()` method

In [11]:
def runningSum(nums):
    return np.array(nums).cumsum().tolist()

In [12]:
%%timeit
assertions()

6.81 µs ± 212 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Simple Iterator approach

In [13]:
def runningSum(nums):
    output = nums[:1]
    for number in nums[1:]:
        output.append(output[-1] + number)
    return output

In [14]:
%%timeit
assertions()

1.84 µs ± 33.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


A more succint, yet slower, one

In [15]:
def runningSum(nums):
    return [sum(nums[:idx+1]) for idx, _ in enumerate(nums)]

In [16]:
%%timeit
assertions()

3.22 µs ± 12.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [17]:
def runningSums(nums):
    for idx, number in enumerate(nums):
        nums[idx] += nums[idx-1]
    return nums

In [18]:
%%timeit
assertions()

3.23 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Two sum (#1)
[Exercise](https://leetcode.com/problems/two-sum)

Given an array of integers, return indices of the two numbers such that they add up to a specific target.

You may assume that each input would have **exactly** one solution, and you may not use the same element twice.

Example:

Given `nums = [2, 7, 11, 15], target = 9`

Because nums[0] + nums[1] = 2 + 7 = 9,  
`return [0, 1]`



In [1]:
def assertions():
    # define checks: list, target, expected output
    checks = (
        ([-3, 4, 3, 90], 0, [0, 2]),
        ([2, 7, 11, 15], 9, [0, 1]),
        ([0, 0, 4, 7], 0, [0, 1]),
        ([3, 2, 4], 6, [1, 2]),
        ([-1, -2, -3, -4, -5], -8, [2, 4]),
    )
    
    failed = False
    for c in checks:
        E = twoSum(c[0], c[1])
        if E != c[2]:
            print(c[0], c[1], E, c[2])
            failed = True
            break
    if failed:
        return

## Numpy approach
* Convert the list into a numpy array to efficiently iterate. 
* Then, for each integer in the array calculate the sum of each element with 
  that integer to check whether any sum adds to the target (winner). 
* Then search those positions in the array, first, the current iteration 
  element.
* Chances are the location returns two positions if the target is the sum
  of the same numbers in different positions in the array. 
* Otherwise, search the position of the second element.   

In [2]:
import numpy as np
def twoSum(nums, target):
    """Return the indices of the integers that sum up to the target."""
    array = np.array(nums)
    for n, i in enumerate(array):
        if (array[n+1:] + i == target).any():
            # There is a winner in this iteration
            first_pos = np.where(array == i)[0]
            if first_pos.size == 2:
                return [first_pos[0], first_pos[1]]
            return [
                first_pos[0], np.where(array == target - i)[0][0]]

In [3]:
%%timeit
assertions()

66 µs ± 371 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Hash table approach (2-loops)
We need a way to get the index of the conjugate and that could be achieved by usign a `dict` where:
* Keys are the values we'll search the conjugate within.
* Values are the current conjugate position
* For any number in the array we'll calculate the conjugate and search in the keys

In [4]:
def twoSum(nums, target):
    indices = {n: idx for idx, n in enumerate(nums)}
    for idx, n in enumerate(nums):
        conjugate = target - n
        if conjugate in indices.keys() and indices[conjugate] != idx:
            return [idx, indices[conjugate]]

In [5]:
%%timeit
assertions()

5.39 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Hash table approach (1-loop)
Update the dict on the fly and look back for the conjugate

In [6]:
def twoSum(nums, target):
    indices = {}
    for idx, n in enumerate(nums):
        conjugate = target - n
        if conjugate in indices.keys():
            return [indices[conjugate], idx]
        indices.update([[n, idx], [n, idx]])

In [7]:
%%timeit
assertions()

6.72 µs ± 96.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Numpy matrix approach
Create a matrix out of the sum of the arrays and look for the target's position. Notice that the matrix created is diagonal so usually `np.where` will return a couple of indexers --the rows and the columms-- with two numbers inside meaning x, y coordinates. In the case of `[0, 0, 4, 7]`, four coordinates will be returned meaning the sum of the submatrix `[[0, 0],[0, 0]` in the top left corner.

This approach greatly exceeded the time limit processing a quite big array.

In [8]:
def twoSum(nums, target):
    n = len(nums)
    matrix = np.full((n, n), nums)
    diagonal = matrix + matrix.T
    # exclude diagonal from the results
    diagonal[np.eye(n).astype(bool)] = target + 1
    find = np.where(diagonal == target)
    if find[0].size == 2:
        return [find[0][0], find[0][1]]
    else:
        return [find[0][1], find[0][2]]

In [9]:
%%timeit
assertions()

74.7 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
